# Florida went red. Where did Trump and Biden gain and lose?

An analysis of Florida voting data to determine what helped make the state go for Donald Trump. We'll look at county voting data and compare changes in margins from 2016 to 2020.

In [1]:
import pandas as pd
import numpy as np

## How did counties vote?
We'll analyze the county results from the state DOE. Find those here: https://floridaelectionwatch.gov/Downloads.

In [2]:
county_data = pd.read_csv('data/2020_county_results.txt', delimiter='\t')
county_data.head()

,ElectionDate,PartyCode,PartyName,RaceCode,RaceName,CountyCode,CountyName,Juris1num,Juris2num,Precincts,PrecinctsReporting,CanNameLast,CanNameFirst,CanNameMiddle,CanVotes
0,11/03/2020,REP,Republican Party,PRE,President of the United States,ALA,Alachua,,,63,0,Pence,Trump,/,50972
1,11/03/2020,REP,Republican Party,PRE,President of the United States,BAK,Baker,,,9,9,Pence,Trump,/,11911
2,11/03/2020,REP,Republican Party,PRE,President of the United States,BAY,Bay,,,44,44,Pence,Trump,/,66097
3,11/03/2020,REP,Republican Party,PRE,President of the United States,BRA,Bradford,,,14,14,Pence,Trump,/,10334
4,11/03/2020,REP,Republican Party,PRE,President of the United States,BRE,Brevard,,,171,171,Pence,Trump,/,207883


In [3]:
# Filter for the presidential race only.
county_pres_data = county_data[county_data["RaceName"] == "President of the United States"]
county_pres_data.head()

,ElectionDate,PartyCode,PartyName,RaceCode,RaceName,CountyCode,CountyName,Juris1num,Juris2num,Precincts,PrecinctsReporting,CanNameLast,CanNameFirst,CanNameMiddle,CanVotes
0,11/03/2020,REP,Republican Party,PRE,President of the United States,ALA,Alachua,,,63,0,Pence,Trump,/,50972
1,11/03/2020,REP,Republican Party,PRE,President of the United States,BAK,Baker,,,9,9,Pence,Trump,/,11911
2,11/03/2020,REP,Republican Party,PRE,President of the United States,BAY,Bay,,,44,44,Pence,Trump,/,66097
3,11/03/2020,REP,Republican Party,PRE,President of the United States,BRA,Bradford,,,14,14,Pence,Trump,/,10334
4,11/03/2020,REP,Republican Party,PRE,President of the United States,BRE,Brevard,,,171,171,Pence,Trump,/,207883


#### Reshape the data so counites have columns for each candidate.

In [4]:
# Pivot candidate rows to columns, preserving the vote counts as values.
fl_county_results = county_pres_data.pivot(index='CountyName', columns='CanNameFirst', values=['CanVotes'])

# Flatten multi-index dataframe
fl_county_results.columns = fl_county_results.columns.get_level_values(1)

# Reset index and make candidate names the headers
fl_county_results.columns = pd.Series(fl_county_results.columns.tolist()).apply(pd.Series).sum(axis=1)
fl_county_results = fl_county_results.reset_index()

fl_county_results.head()

,CountyName,Biden,Blankenship,Carroll,De La Fuente,Hawkins,Howard,Jorgensen,La Riva,Laboch,Simmons,Trump,Walls-Windhauser,Wells
0,Alachua,89704,52,16,70,302,0,1390,94,0,3,50972,0,1
1,Baker,2037,1,0,4,10,0,93,3,0,0,11911,0,0
2,Bay,25614,52,11,54,142,0,1004,45,0,5,66097,0,0
3,Bradford,3160,6,1,7,11,0,110,3,0,0,10334,0,0
4,Brevard,148549,171,36,222,553,0,3178,164,0,8,207883,0,0


#### Get total votes for each county.

In [5]:
fl_county_results['total_votes'] = fl_county_results.iloc[:, 1:].sum(axis=1)
fl_county_results.head()

,CountyName,Biden,Blankenship,Carroll,De La Fuente,Hawkins,Howard,Jorgensen,La Riva,Laboch,Simmons,Trump,Walls-Windhauser,Wells,total_votes
0,Alachua,89704,52,16,70,302,0,1390,94,0,3,50972,0,1,142604
1,Baker,2037,1,0,4,10,0,93,3,0,0,11911,0,0,14059
2,Bay,25614,52,11,54,142,0,1004,45,0,5,66097,0,0,93024
3,Bradford,3160,6,1,7,11,0,110,3,0,0,10334,0,0,13632
4,Brevard,148549,171,36,222,553,0,3178,164,0,8,207883,0,0,360764


In [6]:
# Make a slice of this dataset to narrow down Trump vs. Biden results.
biden_vs_trump = fl_county_results[['CountyName', 'Biden', 'Trump', 'total_votes']]

### Compare data with 2016
Load in results from 2016 and do the same reshaping so we can compare the two elections. Data from 2016 can be manually downloaded here: https://results.elections.myflorida.com/downloadresults.asp?ElectionDate=11/8/2016&DATAMODE=

In [7]:
results_2016 = pd.read_csv('data/11082016Election.txt', delimiter='\t')
results_2016.head(2)

,ElectionDate,PartyCode,PartyName,RaceCode,OfficeDesc,CountyCode,CountyName,Juris1num,Juris2num,Precincts,PrecinctsReporting,CanNameLast,CanNameFirst,CanNameMiddle,CanVotes
0,11/8/2016,REP,Republican,PRE,President of the United States,ALA,Alachua,NaN,NaN,63,63,Trump,Donald,J.,46834
1,11/8/2016,REP,Republican,PRE,President of the United States,BAK,Baker,NaN,NaN,9,9,Trump,Donald,J.,10294


In [8]:
# Reshape using same method as 2020 data, but change relevant fields to match 2016 data format.
county_pres_data_2016 = results_2016[results_2016["OfficeDesc"] == "President of the United States"]

pres_data_2016 = county_pres_data_2016.pivot(index='CountyName', columns='CanNameLast', values=['CanVotes'])
pres_data_2016.columns = pres_data_2016.columns.get_level_values(1)

pres_data_2016.columns = pd.Series(pres_data_2016.columns.tolist()).apply(pd.Series).sum(axis=1)
pres_data_2016 = pres_data_2016.reset_index()
pres_data_2016['total_votes'] = pres_data_2016.iloc[:, 1:].sum(axis=1)

pres_data_2016.head()

,CountyName,Basiago,Castle,Clinton,De La Fuente,Duncan,Fox,Gyurko,Johnson,Kotlikoff,Stein,Trump,Valdivia,total_votes
0,Alachua,1,262,75820,87,1,0,0,4059,0,1507,46834,0,128571
1,Baker,0,23,2112,6,0,0,0,169,0,30,10294,0,12634
2,Bay,0,179,21797,65,0,0,1,2652,0,562,62194,3,87453
3,Bradford,0,33,2924,4,0,0,0,177,0,47,8913,0,12098
4,Brevard,2,744,119679,314,2,0,2,9451,2,2708,181848,0,314752


In [9]:
# Narrow to Clinton vs. Trump data
clinton_vs_trump = pres_data_2016[['CountyName', 'Clinton', 'Trump', 'total_votes']]

### Merge 2020 and 2016 results

In [10]:
results_2020_vs_2016 = biden_vs_trump.merge(clinton_vs_trump, on='CountyName', suffixes=('_2020', '_2016'))
results_2020_vs_2016.head()

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016
0,Alachua,89704,50972,142604,75820,46834,128571
1,Baker,2037,11911,14059,2112,10294,12634
2,Bay,25614,66097,93024,21797,62194,87453
3,Bradford,3160,10334,13632,2924,8913,12098
4,Brevard,148549,207883,360764,119679,181848,314752


#### Calculate vote percentages.

In [11]:
results_2020_vs_2016['Biden_pct'] = results_2020_vs_2016['Biden'] / results_2020_vs_2016['total_votes_2020'] * 100
results_2020_vs_2016['Trump_2020_pct'] = results_2020_vs_2016['Trump_2020'] / results_2020_vs_2016['total_votes_2020'] * 100
results_2020_vs_2016['Clinton_pct'] = results_2020_vs_2016['Clinton'] / results_2020_vs_2016['total_votes_2016'] * 100
results_2020_vs_2016['Trump_2016_pct'] = results_2020_vs_2016['Trump_2016'] / results_2020_vs_2016['total_votes_2016'] * 100
results_2020_vs_2016.head()

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016,Biden_pct,Trump_2020_pct,Clinton_pct,Trump_2016_pct
0,Alachua,89704,50972,142604,75820,46834,128571,62.904266,35.743738,58.971308,36.426566
1,Baker,2037,11911,14059,2112,10294,12634,14.488939,84.721531,16.716796,81.478550
2,Bay,25614,66097,93024,21797,62194,87453,27.534830,71.053707,24.924245,71.117057
3,Bradford,3160,10334,13632,2924,8913,12098,23.180751,75.806925,24.169284,73.673334
4,Brevard,148549,207883,360764,119679,181848,314752,41.176226,57.622989,38.023269,57.775010


#### Flag who won and if a county flipped.

In [12]:
results_2020_vs_2016['2020_winner'] = results_2020_vs_2016.apply(lambda x: 'Trump' if x['Trump_2020_pct'] > x['Biden_pct'] else 'Biden', axis=1)
results_2020_vs_2016['2016_winner'] = results_2020_vs_2016.apply(lambda x: 'Trump' if x['Trump_2016_pct'] > x['Clinton_pct'] else 'Clinton', axis=1)


def check_flips(row):
    if row['2020_winner'] == row['2016_winner']:
        return ''
    elif row['2020_winner'] == 'Biden' and row['2016_winner'] == 'Clinton':
        return ''
    elif row['2020_winner'] == 'Biden' and row['2016_winner'] == 'Trump':
        return 'Biden'
    elif row['2020_winner'] == 'Trump' and row['2016_winner'] == 'Clinton':
        return 'Trump'
    
    
results_2020_vs_2016['flip'] = results_2020_vs_2016.apply(check_flips, axis=1)
results_2020_vs_2016.head()

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016,Biden_pct,Trump_2020_pct,Clinton_pct,Trump_2016_pct,2020_winner,2016_winner,flip
0,Alachua,89704,50972,142604,75820,46834,128571,62.904266,35.743738,58.971308,36.426566,Biden,Clinton,
1,Baker,2037,11911,14059,2112,10294,12634,14.488939,84.721531,16.716796,81.478550,Trump,Trump,
2,Bay,25614,66097,93024,21797,62194,87453,27.534830,71.053707,24.924245,71.117057,Trump,Trump,
3,Bradford,3160,10334,13632,2924,8913,12098,23.180751,75.806925,24.169284,73.673334,Trump,Trump,
4,Brevard,148549,207883,360764,119679,181848,314752,41.176226,57.622989,38.023269,57.775010,Trump,Trump,


### Which counties flipped?
Three Florida counties flipped to Biden, according to the data. Trump won Duval, Pinellas and Seminole in 2016, but not in 2020.

In [13]:
results_2020_vs_2016[results_2020_vs_2016['flip'] != '']

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016,Biden_pct,Trump_2020_pct,Clinton_pct,Trump_2016_pct,2020_winner,2016_winner,flip
14,Duval,252556,233762,492820,205704,211672,432695,51.247108,47.433546,47.540184,48.919447,Biden,Trump,Biden
51,Pinellas,277450,276209,559715,233701,239201,492403,49.569870,49.348150,47.461327,48.578299,Biden,Trump,Biden
56,Seminole,132528,125241,260815,105914,109443,224896,50.813028,48.019094,47.094657,48.663827,Biden,Trump,Biden


## How did county margins change?

In [14]:
margins_data = results_2020_vs_2016
margins_data['Dem_change'] = margins_data['Biden_pct'] - margins_data['Clinton_pct']
margins_data['GOP_change'] = margins_data['Trump_2020_pct'] - margins_data['Trump_2016_pct']

margins_data.to_csv('data/vote_margin_changes.csv', index=False)
margins_data.head()

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016,Biden_pct,Trump_2020_pct,Clinton_pct,Trump_2016_pct,2020_winner,2016_winner,flip,Dem_change,GOP_change
0,Alachua,89704,50972,142604,75820,46834,128571,62.904266,35.743738,58.971308,36.426566,Biden,Clinton,,3.932959,-0.682828
1,Baker,2037,11911,14059,2112,10294,12634,14.488939,84.721531,16.716796,81.478550,Trump,Trump,,-2.227856,3.242981
2,Bay,25614,66097,93024,21797,62194,87453,27.534830,71.053707,24.924245,71.117057,Trump,Trump,,2.610585,-0.063351
3,Bradford,3160,10334,13632,2924,8913,12098,23.180751,75.806925,24.169284,73.673334,Trump,Trump,,-0.988533,2.133590
4,Brevard,148549,207883,360764,119679,181848,314752,41.176226,57.622989,38.023269,57.775010,Trump,Trump,,3.152957,-0.152021


### Where did Biden see the most changes?

#### Biden top margin losses

Biden saw a big loss in Miami-Dade County, as has been reported. He also lost traction in and Osceola, Hendry, Glades and Baker counties compared to Clinton in 2016.

In [15]:
margins_data.sort_values('Dem_change', ascending=True).head()

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016,Biden_pct,Trump_2020_pct,Clinton_pct,Trump_2016_pct,2020_winner,2016_winner,flip,Dem_change,GOP_change
42,Miami-Dade,617864,532833,1156816,624146,333999,980204,53.410741,46.060307,63.675113,34.074438,Biden,Clinton,,-10.264371,11.985869
48,Osceola,97297,73480,172436,85458,50301,140206,56.424992,42.612911,60.951742,35.876496,Biden,Clinton,,-4.526750,6.736415
24,Hendry,4929,7906,12940,4615,6195,11115,38.091190,61.097372,41.520468,55.735493,Trump,Trump,,-3.429278,5.361880
20,Glades,1385,3782,5193,1271,2996,4353,26.670518,72.828808,29.198254,68.826097,Trump,Trump,,-2.527736,4.002711
1,Baker,2037,11911,14059,2112,10294,12634,14.488939,84.721531,16.716796,81.478550,Trump,Trump,,-2.227856,3.242981


#### Biden top margin gains

Biden managed to make minor improvements to the Democratic vote share compared to Clinton in Okaloosa, Santa Rosa, Clay, St. Johns and Escambia counties.

In [16]:
margins_data.sort_values('Dem_change', ascending=False).head()

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016,Biden_pct,Trump_2020_pct,Clinton_pct,Trump_2016_pct,2020_winner,2016_winner,flip,Dem_change,GOP_change
45,Okaloosa,34248,79798,116373,23780,71893,100855,29.429507,68.570888,23.578405,71.283526,Trump,Trump,,5.851102,-2.712637
54,Santa Rosa,27612,77385,106928,18464,65339,87745,25.822984,72.371128,21.042794,74.464642,Trump,Trump,,4.780189,-2.093514
9,Clay,38317,84480,124405,27822,74963,106483,30.800209,67.907238,26.128114,70.399031,Trump,Trump,,4.672095,-2.491792
57,St. Johns,63850,110946,176622,43099,88684,136514,36.150649,62.815504,31.571121,64.963300,Trump,Trump,,4.579528,-2.147796
15,Escambia,70929,96674,170372,57461,88808,152469,41.631841,56.742892,37.687005,58.246594,Trump,Trump,,3.944836,-1.503702


#### Count how many counties Biden saw gains and losses in.

In [17]:
biden_gains_count = margins_data[margins_data["Dem_change"] > 0]
biden_loss_count = margins_data[margins_data["Dem_change"] < 0]
print("Gains:", biden_gains_count.shape[0])
print("Losses:", biden_loss_count.shape[0])

Gains: 43
Losses: 24


### Where did Trump see the most changes?

#### Trump top margin losses

Compared to Biden, Trump saw much less of a decline in support in counties. The biggest loss is in Okaloosa, where he won 2.7 points less this election than he did in 2016.

In [18]:
margins_data.sort_values('GOP_change', ascending=True).head()

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016,Biden_pct,Trump_2020_pct,Clinton_pct,Trump_2016_pct,2020_winner,2016_winner,flip,Dem_change,GOP_change
45,Okaloosa,34248,79798,116373,23780,71893,100855,29.429507,68.570888,23.578405,71.283526,Trump,Trump,,5.851102,-2.712637
9,Clay,38317,84480,124405,27822,74963,106483,30.800209,67.907238,26.128114,70.399031,Trump,Trump,,4.672095,-2.491792
57,St. Johns,63850,110946,176622,43099,88684,136514,36.150649,62.815504,31.571121,64.963300,Trump,Trump,,4.579528,-2.147796
54,Santa Rosa,27612,77385,106928,18464,65339,87745,25.822984,72.371128,21.042794,74.464642,Trump,Trump,,4.780189,-2.093514
15,Escambia,70929,96674,170372,57461,88808,152469,41.631841,56.742892,37.687005,58.246594,Trump,Trump,,3.944836,-1.503702


#### Trump top margin gains

Trump's margin gains also outweigh the improvements Biden made. Trump managed to improve his vote share gain by nearly 12 points in Miami-Dade counties, while also upping his margins in Osceola and Hendry counties.

In [19]:
margins_data.sort_values('GOP_change', ascending=False).head()

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016,Biden_pct,Trump_2020_pct,Clinton_pct,Trump_2016_pct,2020_winner,2016_winner,flip,Dem_change,GOP_change
42,Miami-Dade,617864,532833,1156816,624146,333999,980204,53.410741,46.060307,63.675113,34.074438,Biden,Clinton,,-10.264371,11.985869
48,Osceola,97297,73480,172436,85458,50301,140206,56.424992,42.612911,60.951742,35.876496,Biden,Clinton,,-4.526750,6.736415
24,Hendry,4929,7906,12940,4615,6195,11115,38.091190,61.097372,41.520468,55.735493,Trump,Trump,,-3.429278,5.361880
6,Calhoun,1209,5274,6526,1241,4655,6081,18.525896,80.815201,20.407828,76.549910,Trump,Trump,,-1.881931,4.265291
20,Glades,1385,3782,5193,1271,2996,4353,26.670518,72.828808,29.198254,68.826097,Trump,Trump,,-2.527736,4.002711


#### Count how many counties Trump saw gains and losses in.

In [20]:
trump_gains_count = margins_data[margins_data['GOP_change'] > 0]
trump_loss_count = margins_data[margins_data['GOP_change'] < 0]
print('Gains:', trump_gains_count.shape[0])
print('Losses:', trump_loss_count.shape[0])

Gains: 49
Losses: 18


### Margins of victory
Now, compare overall margin of victory changes by calculating the point differences between the candidates' vote percentages.

In [21]:
margins_data['2016_mov'] = abs(margins_data['Clinton_pct'] - margins_data['Trump_2016_pct'])
margins_data['2020_mov'] = abs(margins_data['Biden_pct'] - margins_data['Trump_2020_pct'])
margins_data['mov_change'] = margins_data['2020_mov'] - margins_data['2016_mov']

# Save data
margins_data.to_csv('data/mov_changes.csv', index=False)

#### Trump's margin of victory gains
Trump improved his margin of victory most in Hendry County followed by Glades, Calhoun, Baker and Okeechobee counties.

In [22]:
trump_wins = margins_data[margins_data['2020_winner'] == 'Trump']
trump_wins.sort_values('mov_change', ascending=False).head()

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016,Biden_pct,Trump_2020_pct,Clinton_pct,Trump_2016_pct,2020_winner,2016_winner,flip,Dem_change,GOP_change,2016_mov,2020_mov,mov_change
24,Hendry,4929,7906,12940,4615,6195,11115,38.091190,61.097372,41.520468,55.735493,Trump,Trump,,-3.429278,5.361880,14.215025,23.006182,8.791158
20,Glades,1385,3782,5193,1271,2996,4353,26.670518,72.828808,29.198254,68.826097,Trump,Trump,,-2.527736,4.002711,39.627843,46.158290,6.530447
6,Calhoun,1209,5274,6526,1241,4655,6081,18.525896,80.815201,20.407828,76.549910,Trump,Trump,,-1.881931,4.265291,56.142082,62.289304,6.147222
1,Baker,2037,11911,14059,2112,10294,12634,14.488939,84.721531,16.716796,81.478550,Trump,Trump,,-2.227856,3.242981,64.761754,70.232591,5.470837
46,Okeechobee,4390,11470,15954,3959,9356,13653,27.516610,71.894196,28.997290,68.527064,Trump,Trump,,-1.480680,3.367132,39.529774,44.377586,4.847812


#### Trump's margin of victory losses
Trump's margin of victory shrunk most in Okaloosa County, followed by Clay, Santa Rosa, St. Johns and Escambia counties.

In [35]:
trump_wins.sort_values('mov_change', ascending=True).head()

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016,Biden_pct,Trump_2020_pct,Clinton_pct,Trump_2016_pct,2020_winner,2016_winner,flip,Dem_change,GOP_change,2016_mov,2020_mov,mov_change
45,Okaloosa,34248,79798,116373,23780,71893,100855,29.429507,68.570888,23.578405,71.283526,Trump,Trump,,5.851102,-2.712637,47.705121,39.141382,-8.563740
9,Clay,38317,84480,124405,27822,74963,106483,30.800209,67.907238,26.128114,70.399031,Trump,Trump,,4.672095,-2.491792,44.270916,37.107029,-7.163887
54,Santa Rosa,27612,77385,106928,18464,65339,87745,25.822984,72.371128,21.042794,74.464642,Trump,Trump,,4.780189,-2.093514,53.421847,46.548145,-6.873703
57,St. Johns,63850,110946,176622,43099,88684,136514,36.150649,62.815504,31.571121,64.963300,Trump,Trump,,4.579528,-2.147796,33.392180,26.664855,-6.727325
15,Escambia,70929,96674,170372,57461,88808,152469,41.631841,56.742892,37.687005,58.246594,Trump,Trump,,3.944836,-1.503702,20.559589,15.111051,-5.448538


#### Count how many counties Trump had gains and losses of victory margins.

In [36]:
trump_mov_gains_count = trump_wins[trump_wins['mov_change'] > 0]
trump_mov_loss_count = trump_wins[trump_wins['mov_change'] < 0]
print('Gains:', trump_mov_gains_count.shape[0])
print('Losses:', trump_mov_loss_count.shape[0])

Gains: 33
Losses: 22


#### Bidens's margin of victory gains
Biden upped his margin of victory compared to Clinton most in Alachua County, followed by Leon, Duval, Seminole and Hillsborough counties.

In [37]:
biden_wins = margins_data[(margins_data['2020_winner'] == 'Biden') & (margins_data['flip'] == '')]
biden_wins.sort_values('mov_change', ascending=False).head()

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016,Biden_pct,Trump_2020_pct,Clinton_pct,Trump_2016_pct,2020_winner,2016_winner,flip,Dem_change,GOP_change,2016_mov,2020_mov,mov_change
0,Alachua,89704,50972,142604,75820,46834,128571,62.904266,35.743738,58.971308,36.426566,Biden,Clinton,,3.932959,-0.682828,22.544742,27.160528,4.615787
35,Leon,103517,57453,162929,92068,53821,152132,63.535037,35.262599,60.518497,35.377830,Biden,Clinton,,3.016540,-0.115231,25.140667,28.272438,3.131770
27,Hillsborough,376367,327398,712063,307896,266870,597660,52.855857,45.978797,51.516916,44.652478,Biden,Clinton,,1.338941,1.326319,6.864438,6.877060,0.012622
18,Gadsden,16153,7465,23762,15020,6728,22113,67.978285,31.415706,67.923846,30.425542,Biden,Clinton,,0.054439,0.990164,37.498304,36.562579,-0.935725
47,Orange,395014,245398,647403,329894,195216,546275,61.015164,37.904983,60.389730,35.735847,Biden,Clinton,,0.625433,2.169136,24.653883,23.110180,-1.543703


#### Bidens's margin of victory losses
Biden took big losses to his margin of victory, compared to Clinton, in Miami-Dade County, followed by Osceola, Broward, Palm Beach and Orange counties.

In [26]:
biden_wins.sort_values('mov_change', ascending=True).head()

,CountyName,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016,Biden_pct,Trump_2020_pct,Clinton_pct,Trump_2016_pct,2020_winner,2016_winner,flip,Dem_change,GOP_change,2016_mov,2020_mov,mov_change
42,Miami-Dade,617864,532833,1156816,624146,333999,980204,53.410741,46.060307,63.675113,34.074438,Biden,Clinton,,-10.264371,11.985869,29.600675,7.350434,-22.250241
48,Osceola,97297,73480,172436,85458,50301,140206,56.424992,42.612911,60.951742,35.876496,Biden,Clinton,,-4.526750,6.736415,25.075246,13.812081,-11.263165
5,Broward,618752,333409,957672,553320,260951,831951,64.610013,34.814529,66.508725,31.366150,Biden,Clinton,,-1.898712,3.448379,35.142575,29.795483,-5.347091
49,Palm Beach,433572,334711,773158,374673,272402,662332,56.078059,43.291410,56.568760,41.127712,Biden,Clinton,,-0.490701,2.163698,15.441048,12.786649,-2.654399
47,Orange,395014,245398,647403,329894,195216,546275,61.015164,37.904983,60.389730,35.735847,Biden,Clinton,,0.625433,2.169136,24.653883,23.110180,-1.543703


In [27]:
biden_mov_gains_count = biden_wins[biden_wins['mov_change'] > 0]
biden_mov_loss_count = biden_wins[biden_wins['mov_change'] < 0]
print('Gains:', biden_mov_gains_count.shape[0])
print('Losses:', biden_mov_loss_count.shape[0])

Gains: 3
Losses: 6


## Statewide margins

Trump won Florida in 2016 and 2020. Let's take a look at how his margins of victory compare.

In [38]:
statewide_results = results_2020_vs_2016[['Biden','Trump_2020','total_votes_2020','Clinton','Trump_2016','total_votes_2016']]
statewide_results = statewide_results.agg(['sum'])

statewide_results.head()

,Biden,Trump_2020,total_votes_2020,Clinton,Trump_2016,total_votes_2016
sum,5297045,5668731,11067456,4504975,4617886,9420039


In [39]:
statewide_results['2020_Trump_pct'] = statewide_results['Trump_2020'] / statewide_results['total_votes_2020'] * 100
statewide_results['2020_Biden_pct'] = statewide_results['Biden'] / statewide_results['total_votes_2020'] * 100
statewide_results['2016_Clinton_pct'] = statewide_results['Clinton'] / statewide_results['total_votes_2016'] * 100
statewide_results['2016_Trump_pct'] = statewide_results['Trump_2016'] / statewide_results['total_votes_2016'] * 100
statewide_results['2016_Trump_pct'][0]

print("Trump's 2016 margin of victory:", statewide_results['2016_Trump_pct'][0] - statewide_results['2016_Clinton_pct'][0])
print("Trump's 2020 margin of victory:", statewide_results['2020_Trump_pct'][0] - statewide_results['2020_Biden_pct'][0])


Trump's 2016 margin of victory: 1.1986256107856832
Trump's 2020 margin of victory: 3.3583688970618013
